In [1]:
import math
from camel.toolkits import FunctionTool

def mathtool(x:float)->float:
    """计算一个数的平方根

    Args:
        x (float): 输入的数

    Returns:
        float: 返回该数的平方根
    """
    return math.sqrt(x)
mathtool(4)

2.0

In [2]:
mathtool = FunctionTool(mathtool)

print(mathtool.get_function_description())
print(mathtool.get_function_name())

计算一个数的平方根
mathtool


In [ ]:
from camel.agents import ChatAgent
from camel.models import ModelFactory
from camel.types import ModelPlatformType

import os
from dotenv import load_dotenv,find_dotenv
import math

load_dotenv(find_dotenv())

# 定义系统消息
sys_msg = "你是一个数学大师，擅长各种数学问题。当你遇到数学问题的时候，你要调用工具，将工具计算的结果作为答案"

# 初始化agent
api_key = os.getenv('ModelFactory_Key')

model = ModelFactory.create(
    model_platform=ModelPlatformType.OPENAI_COMPATIBLE_MODEL,
    model_type="Qwen/Qwen2.5-72B-Instruct",
    url='https://api-inference.modelscope.cn/v1/',
    api_key=api_key
)

# 创建agent并添加工具
agent = ChatAgent(
    system_message=sys_msg,
    model=model,
    output_language='中文',
    tools=[]
)

# 定义用户消息
usr_msg = "2的平方根是多少？"
# response = agent.step(usr_msg)
# print(response.msgs[0].content)

In [14]:
from camel.toolkits import MathToolkit,SearchToolkit
from camel.types.agents import ToolCallingRecord
from camel.models import ModelFactory
from camel.types import ModelPlatformType
from camel.societies import RolePlaying
from camel.utils import print_text_animated
from toolkit.MySearchToolkit import MySearchToolkit

from colorama import Fore

import os
from dotenv import load_dotenv,find_dotenv

load_dotenv(find_dotenv())
api_key = os.getenv('ModelFactory_Key')
# 设置代理
# os.environ["http_proxy"] = "http://127.0.0.1:7897"
# os.environ["https_proxy"] = "http://127.0.0.1:7897"

# 定义工具包

# tools_list = [
#     *MySearchToolkit().get_tools(),
#     *MathToolkit().get_tools()
# ]

# 只使用DuckDuckGo搜索情况下使用下面的工具包即可

tools_list = [
    # MySearchToolkit().search_duckduckgo,
    *SearchToolkit().get_tools(),
    *MathToolkit().get_tools()
]


# 设置任务
# task_prompt = ("假设现在是2024年，"
#                "牛津大学的成立年份，并计算出其当前年龄。"
#                "然后再将这个年龄加上10年。使用百度搜索工具。")

# task_prompt = ("假设现在是2024年，"
#         "牛津大学的成立年份，并计算出其当前年龄。"
#         "然后再将这个年龄加上10年。使用谷歌搜索工具。")

# 如果没有谷歌搜索API，可以使用duckduckgo工具，无需设置api即可使用
task_prompt = ("假设现在是2024年，"
        "牛津大学的成立年份，并计算出其当前年龄。"
        "然后再将这个年龄加上10年。使用duckduckgo搜索工具。")

# 创建模型

model = ModelFactory.create(
    model_platform=ModelPlatformType.OPENAI_COMPATIBLE_MODEL,
    model_type="Qwen/Qwen2.5-72B-Instruct",
    url='https://api-inference.modelscope.cn/v1/',
    api_key= os.getenv('ModelFactory_Key')
)

# 设置角色扮演
role_play_session = RolePlaying(
    assistant_role_name="搜索者",
    user_role_name="教授",
    assistant_agent_kwargs=dict(
        model=model,
        tools=tools_list,
    ),
    user_agent_kwargs=dict(
        model=model,
    ),
    task_prompt=task_prompt,
    with_task_specify=False,
    output_language='中文'
)

# 设置聊天轮次限制
chat_turn_limit = 10

print(
    Fore.GREEN
    + f"AI助手系统消息:\n{role_play_session.assistant_sys_msg}\n"
)
print(
    Fore.BLUE + f"AI用户系统消息:\n{role_play_session.user_sys_msg}\n"
)

print(Fore.YELLOW + f"原始任务提示:\n{task_prompt}\n")
print(
    Fore.CYAN
    + "指定的任务提示:"
    + f"\n{role_play_session.specified_task_prompt}\n"
)
print(Fore.RED + f"最终任务提示:\n{role_play_session.task_prompt}\n")

AI助手系统消息:
BaseMessage(role_name='搜索者', role_type=<RoleType.ASSISTANT: 'assistant'>, meta_dict={'task': '假设现在是2024年，牛津大学的成立年份，并计算出其当前年龄。然后再将这个年龄加上10年。使用duckduckgo搜索工具。', 'assistant_role': '搜索者', 'user_role': '教授'}, content='===== RULES OF ASSISTANT =====\nNever forget you are a 搜索者 and I am a 教授. Never flip roles! Never instruct me!\nWe share a common interest in collaborating to successfully complete a task.\nYou must help me to complete the task.\nHere is the task: 假设现在是2024年，牛津大学的成立年份，并计算出其当前年龄。然后再将这个年龄加上10年。使用duckduckgo搜索工具。. Never forget our task!\nI must instruct you based on your expertise and my needs to complete the task.\n\nI must give you one instruction at a time.\nYou must write a specific solution that appropriately solves the requested instruction and explain your solutions.\nYou must decline my instruction honestly if you cannot perform the instruction due to physical, moral, legal reasons or your capability and explain the reasons.\nUnless I say the task is completed, y

In [15]:

n = 0
input_msg = role_play_session.init_chat()
while n < chat_turn_limit:
    n += 1
    assistant_response, user_response = role_play_session.step(input_msg)

    if assistant_response.terminated:
        print(
            Fore.GREEN
            + (
                "AI助手终止。原因: "
                f"{assistant_response.info['termination_reasons']}."
            )
        )
        break
    if user_response.terminated:
        print(
            Fore.GREEN
            + (
                "AI用户终止。"
                f"原因: {user_response.info['termination_reasons']}."
            )
        )
        break

    # 打印用户的输出
    print_text_animated(
        Fore.BLUE + f"AI用户:\n\n{user_response.msg.content}\n"
    )

    if "CAMEL_TASK_DONE" in user_response.msg.content:
        break

    # 打印助手的输出，包括任何函数执行信息
    print_text_animated(Fore.GREEN + "AI助手:")
    tool_calls: list[ToolCallingRecord] = assistant_response.info[
        'tool_calls'
    ]
    for func_record in tool_calls:
        print_text_animated(f"{func_record}")
    print_text_animated(f"{assistant_response.msg.content}\n")

    input_msg = assistant_response.msg

2025-08-31 16:46:57,130 - root - WARNING - Error executing tool 'search_duckduckgo': Execution of function search_duckduckgo failed with arguments () and {'query': '牛津大学 成立年份', 'source': 'text', 'max_results': 1}. Error: https://duckduckgo.com/?q=%E7%89%9B%E6%B4%A5%E5%A4%A7%E5%AD%A6+%E6%88%90%E7%AB%8B%E5%B9%B4%E4%BB%BD 202 Ratelimit
2025-08-31 16:47:00,270 - root - WARNING - Error executing tool 'search_google': Execution of function search_google failed with arguments () and {'query': '牛津大学 成立年份', 'num_result_pages': 1}. Error: Missing or empty required API keys in environment variables: GOOGLE_API_KEY, SEARCH_ENGINE_ID.
You can obtain the API key from the official website
AI用户:

Instruction: 使用DuckDuckGo搜索牛津大学的成立年份。
Input: None
AI助手:Tool Execution: search_duckduckgo
	Args: {'query': '牛津大学 成立年份', 'source': 'text', 'max_results': 1}
	Result: {'error': "Error executing tool 'search_duckduckgo': Execution of function search_duckduckgo failed with arguments () and {'query': '牛津大学 成立年份', '